In [1]:
import pretty_midi_fluidsynth as pmf
import librosa
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio

from PIL import Image
import soundfile as sf

In [2]:
sr = 44100

# hop_len = 512
# freq_w = 3
# octave = 8

fs = 100

# gen data save

In [3]:
import os
from pathlib import Path
from tqdm import tqdm

In [4]:
x_path = Path('./gen_data/wav')
x_path.mkdir(parents=True, exist_ok=True)

y_path = Path('./gen_data/midi')
y_path.mkdir(parents=True, exist_ok=True)

In [5]:
sf_list = os.listdir("./sf")
print(sf_list)

['FluidR3Mono_GM.sf3', 'GeneralUser GS 1.35.sf2', 'GeneralUser GS Live-Audigy v1.44.sf2', 'GeneralUser GS MuseScore v1.442.sf2', 'GeneralUser GS SoftSynth v1.44.sf2', 'GeneralUser GS v1.471.sf2', 'MuseScore_General.sf3', 'TimGM6mb.sf2', 'weedsgm4_update.sf2']


In [6]:
for sf_file in sf_list:
    print(sf_file.split('.')[0])

FluidR3Mono_GM
GeneralUser GS 1
GeneralUser GS Live-Audigy v1
GeneralUser GS MuseScore v1
GeneralUser GS SoftSynth v1
GeneralUser GS v1
MuseScore_General
TimGM6mb
weedsgm4_update


In [7]:
for sf_file in sf_list[:]:
    for idx in tqdm(range(10000)):
        instr_list = []

        n = np.random.randint(1,10)
        for _ in range(n):
            i = np.random.randint(128)
            instr = pmf.Instrument(program=i)
            instr.name = pmf.program_to_instrument_name(i)

            note_n = np.random.randint(2,10)
            velocity = np.random.randint(50,125)
            
            for _ in range(note_n):
                pitch = np.random.randint(24,120)
                start_time = np.random.randint(1,10)
                end_time = start_time + np.random.randint(1,4)
                note = pmf.Note(velocity=velocity,
                               pitch=pitch,
                               start=start_time,
                               end=end_time,)
                instr.notes.append(note)
            instr_list.append(instr)

        pm = pmf.PrettyMIDI()
        for instr in instr_list:
            pm.instruments.append(instr)

        sf_path = os.path.join('./sf/',sf_file)
        x = pm.fluidsynth(sr,sf2_path=sf_path)

        filename = f"{sf_file.split('.')[0]}-{idx:04d}"
        
        midi_path = os.path.join(y_path,f'{filename}.midi')
        pm.write(midi_path)
        
#         y = pm.get_piano_roll(fs).astype(np.uint8)
#         img = Image.fromarray(y)
#         img_path = os.path.join(y_path,f'{filename}.png')
#         img.save(img_path)
        
        wav_path = os.path.join(x_path,f'{filename}.wav')
        sf.write(wav_path,x,sr,'PCM_32')

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [46:53<00:00,  3.55it/s]


In [8]:
print("completed")

completed
